# Problem 49
Sudoku is a puzzle where you're given a partially-filled 9 by 9 grid with digits. The objective is to fill the grid with the constraint that every row, column, and box (3 by 3 subgrid) must contain all of the digits from 1 to 9.

Implement an efficient sudoku solver.

---
## Solution

In [231]:
# solution code - board setup
import random

def print_board(board):
    str_board = ""
    for i in range(len(board)):
        for j in range(len(board[i])):
                str_board += f"{board[i][j]}  "
        str_board += "\n"
    return print(str_board)
        
        
def board_assembly(difficulty = "Easy"):
    board = []
    for i in range(9):
        row = []
        for j in range(9):
            row.append('')
        board.append(row)
    
    # Fill in the board using backtracking
    backtrack(0, 0, board)
    
    # Remove numbers based on difficulty
    if difficulty == "Easy":
        num_to_remove = 15
    elif difficulty == "Medium":
        num_to_remove = 40
    elif difficulty == "Hard":
        num_to_remove = 70
    
    remove_count = 0
    while remove_count < num_to_remove:
        row = random.randint(0, 8)
        col = random.randint(0, 8)
        if board[row][col] != '':
            temp = board[row][col]
            board[row][col] = ''
            
            # Check if there is a unique solution
            num_solutions = count_solutions(board)
            if num_solutions != 1:
                board[row][col] = temp
            else:
                remove_count += 1
            
    return board


def backtrack(row, col, board):
    if row == 9:
        return True
    
    next_row, next_col = get_next_position(row, col)
    
    if board[row][col] != '':
        return backtrack(next_row, next_col, board)
    
    nums = list(map(str, range(1, 10)))
    random.shuffle(nums)
    
    for num in nums:
        if is_valid_move(row, col, num, board):
            board[row][col] = num
            if backtrack(next_row, next_col, board):
                return True
            board[row][col] = ''
    
    return False


def get_next_position(row, col):
    if col == 8:
        return row + 1, 0
    else:
        return row, col + 1


def is_valid_move(row, col, num, board):
    # Check row
    for i in range(9):
        if board[row][i] == num:
            return False
    
    # Check column
    for i in range(9):
        if board[i][col] == num:
            return False
    
    # Check square
    square_row = (row // 3) * 3
    square_col = (col // 3) * 3
    for i in range(square_row, square_row + 3):
        for j in range(square_col, square_col + 3):
            if board[i][j] == num:
                return False
    
    return True


def count_solutions(board):
    # Find first empty cell
    row, col = find_empty_cell(board)
    
    if row == -1 and col == -1:
        # Board is complete
        return 1
    
    count = 0
    nums = list(map(str, range(1, 10)))
    random.shuffle(nums)
    
    for num in nums:
        if is_valid_move(row, col, num, board):
            board[row][col] = num
            count += count_solutions(board)
            board[row][col] = '*'
    
    return count


def find_empty_cell(board):
    for i in range(9):
        for j in range(9):
            if board[i][j] == '':
                return i, j
    return -1, -1

In [232]:
import numpy as np

def box_maker():
    # create 3x3 grid of tiles
    tiles = np.array([(i,j) for i in range(0,3) for j in range(0,3)])
    # add additional rows and columns to each box
    add_on = np.array([3*i for i in range(0,3)])
    col_boxes = [tiles + np.array([0,ao]) for ao in add_on]
    boxes = []
    for box in col_boxes:
        add_on = 0
        for i in range(3):
            new = []
            for tile in box:
                new.append([tile[0]+add_on, tile[1]])
            boxes.append(new)
            add_on += 3
    boxes.sort(key = lambda x:x[0][1])
    return boxes    


def check_board(board):
    list_checks = []
    # columns
    list_checks += [[[j, i] for i in range(len(board))] for j in range(len(board))]
    # rows
    list_checks += [[[i, j] for i in range(len(board))] for j in range(len(board))]
    # boxes
    list_checks += box_maker()
    for l in list_checks:
        results = [board[tile[0]][tile[1]] for tile in l]
        if("*" in results):
            return False
        results.sort()
        if(results != [str(i) for i in range(1,10)]):
            return False
    return True


def row_solve(board, change):
    integers = ['1','2','3','4','5','6','7','8','9']
    for i in range(len(board)):
            int_join = [j for j in board[i] if j != "*"]
            if(len(int_join) == 8):
                missing_index = board[i].index("*")
                missing_num = set(integers) - set(int_join)
                board[i][missing_index] = list(missing_num)[0]
                change = True
    return board, change


def column_solver(board, change):
    integers = ['1','2','3','4','5','6','7','8','9']
    for i in range(len(board)):
            column = []
            missing_row, missing_col = 0, 0
            miss_count = 0
            for j in range(len(board)):
                column.append(board[j][i])
                if(board[j][i] == "*"):
                    miss_count += 1
                    if(miss_count == 2):
                        break
                    missing_row, missing_col = j, i
            if(miss_count == 1):
                int_join = [j for j in column if j != "*"]
                if(len(int_join) == 8):
                    missing_num = set(integers) - set(int_join)
                    board[missing_row][missing_col] = list(missing_num)[0]
                    change = True
    return board, change


def box_solver(boxes, board, change):
    integers = ['1','2','3','4','5','6','7','8','9']
    for box in boxes:
            miss_count = 0
            current_ints = []
            missing_row, missing_col = 0, 0
            for tile in box:
                current_tile = board[tile[0]][tile[1]]
                if(current_tile == "*"):
                    miss_count += 1
                    missing_row, missing_col = tile[0], tile[1]
                else:
                    current_ints.append(current_tile)
                if(miss_count > 1):
                    break
            if(miss_count == 1):
                if(len(current_ints) == 8):
                    missing_num = set(integers) - set(current_ints)
                    board[missing_row][missing_col] = list(missing_num)[0]
                    change = True
    return board, change


def hidden_single(boxes, board, change):
    integers = ['1', '2', '3', '4', '5', '6', '7', '8', '9']
    
    for box in boxes:
        missing_tiles = [(i, j) for i, j in box if board[i][j] == "*"]
        for i, j in missing_tiles:
            row_tiles = [board[i][col] for col in range(9) if board[i][col] != "*"]
            col_tiles = [board[row][j] for row in range(9) if board[row][j] != "*"]
            box_tiles = [board[x][y] for x, y in box if board[x][y] != "*"]
            
            candidates = set(integers) - set(row_tiles) - set(col_tiles) - set(box_tiles)
            if len(candidates) == 1:
                board[i][j] = candidates.pop()
                change = True
                
    return board, change

# solution code - solver
def soduko_solver(board):
    boxes = box_maker()
    change = True
    while(change):
        change = False

        # row solve
        board, change = row_solve(board, change)
        if(check_board(board) == True): break

        # column solve
        board, change = column_solver(board, change)
        if(check_board(board) == True): break

        # box solve
        board, change = box_solver(boxes, board, change)
        if(check_board(board) == True): break

        # hidden single
        board, change = hidden_single(boxes, board, change)
        if(check_board(board) == True): break

    result = ''
    if(check_board(board) == True):
        result += "Board Solved!!"
    else:
        result += "Board not solved :("

    return board, result

---
## Test Cases

In [233]:
# solution testing test cases

# setup board
# mode options: "Easy", "Medium", "Hard"
mode = "Easy"
board = board_assembly(difficulty = mode)
print_board(board)
board, result = soduko_solver(board)
print(result)
print_board(board)


*  7  8  5  *  4  *  1  2  
1  3  *  6  9  2  4  8  7  
6  2  4  8  1  7  9  3  5  
5  *  1  2  *  9  *  7  6  
4  9  7  3  5  6  1  *  8  
3  6  2  *  7  8  5  4  9  
2  4  9  7  6  1  8  *  *  
7  5  6  4  8  3  *  9  1  
8  1  3  9  *  5  7  6  4  

Board Solved!!
9  7  8  5  3  4  6  1  2  
1  3  5  6  9  2  4  8  7  
6  2  4  8  1  7  9  3  5  
5  8  1  2  4  9  3  7  6  
4  9  7  3  5  6  1  2  8  
3  6  2  1  7  8  5  4  9  
2  4  9  7  6  1  8  5  3  
7  5  6  4  8  3  2  9  1  
8  1  3  9  2  5  7  6  4  



In [234]:
mode = "Medium"
board = board_assembly(difficulty = mode)
print_board(board)
board, result = soduko_solver(board)
print(result)
print_board(board)

3  *  *  8  6  *  *  2  *  
1  6  *  3  9  *  4  5  8  
8  4  7  *  *  5  9  3  6  
*  7  8  2  5  *  1  *  3  
5  1  *  *  4  3  8  9  2  
*  2  3  6  8  1  5  *  4  
*  *  *  4  *  2  *  8  9  
2  3  9  5  1  8  6  *  *  
*  8  4  *  3  6  *  *  *  

Board Solved!!
3  9  5  8  6  4  7  2  1  
1  6  2  3  9  7  4  5  8  
8  4  7  1  2  5  9  3  6  
4  7  8  2  5  9  1  6  3  
5  1  6  7  4  3  8  9  2  
9  2  3  6  8  1  5  7  4  
6  5  1  4  7  2  3  8  9  
2  3  9  5  1  8  6  4  7  
7  8  4  9  3  6  2  1  5  



In [235]:
mode = "Hard"
board = board_assembly(difficulty = mode)
print_board(board)
board, result = soduko_solver(board)
print(result)
print_board(board)

1  7  9  *  2  5  *  *  3  
*  3  5  *  4  1  7  *  *  
8  4  2  *  *  7  1  *  *  
4  8  *  *  1  3  *  5  *  
7  *  3  4  5  *  2  6  *  
*  *  *  *  *  6  3  8  *  
*  *  8  1  *  4  *  7  2  
*  5  7  6  *  2  *  *  9  
*  *  *  *  7  *  8  3  6  

Board Solved!!
1  7  9  8  2  5  6  4  3  
6  3  5  9  4  1  7  2  8  
8  4  2  3  6  7  1  9  5  
4  8  6  2  1  3  9  5  7  
7  9  3  4  5  8  2  6  1  
5  2  1  7  9  6  3  8  4  
9  6  8  1  3  4  5  7  2  
3  5  7  6  8  2  4  1  9  
2  1  4  5  7  9  8  3  6  



---
## Solution Explained

### soduko_solver(board) solution

<br>

#### Board setup functions

1. `print_board(board)`: This function takes a board list as input and prints it to the console in a visually appealing format. It loops through each row and column of the board list and concatenates the elements of the row into a string with spaces in between each element. It then appends a newline character at the end of each row to print the rows on separate lines. The function returns the output of the print function.

2. `board_assembly(difficulty = "Easy")`: This function creates and returns a 9x9 Sudoku board list with a specified difficulty level. It initializes an empty board list with 81 empty strings. It then uses the backtrack() function to fill in the board list with numbers using backtracking. Finally, it removes a specified number of numbers from the board list based on the specified difficulty level.

3. `backtrack(row, col, board)`: This function recursively fills in a 9x9 Sudoku board list with numbers using backtracking. It takes the current row, col, and board list as input. If it has reached the end of the board list, it returns True. Otherwise, it gets the next empty cell using the get_next_position() function. If the current cell is already filled, it continues to the next cell using recursion. If the current cell is empty, it shuffles the numbers 1 through 9 and tries each number in a random order. If a number is valid, it inserts it into the current cell and continues to the next cell using recursion. If no number is valid, it backtracks by removing the inserted number from the current cell and trying the next number. If no number is valid for any cell in the current branch of recursion, it returns False.

4. `get_next_position(row, col)`: This function takes the current row and col as input and returns the coordinates of the next cell in the board list that is not filled. If the current cell is in the last column, it returns the first cell of the next row. Otherwise, it returns the next column in the same row.

5. `is_valid_move(row, col, num, board)`: This function checks if a given num can be legally placed in the cell at row and col in the board list. It first checks if the num is already in the same row or column as the current cell. If so, it returns False. It then checks if the num is already in the same 3x3 square as the current cell. If so, it returns False. Otherwise, it returns True.

6. `count_solutions(board)`: This function counts the number of solutions to a given board list by recursively trying to fill in the empty cells using backtracking. It first finds the first empty cell using the find_empty_cell() function. If there are no more empty cells, it returns 1 to indicate that a solution has been found. Otherwise, it shuffles the numbers 1 through 9 and tries each number in a random order. If a number is valid, it inserts it into the empty cell and continues to the next empty cell using recursion. If no number is valid, it backtracks by removing the inserted number from the current cell and trying the next number. It repeats this process until all solutions have been counted.

7. `find_empty_cell(board)`: This function iterates over each cell in the board using nested for-loops and checks if the value of the current cell is an empty string. If the current cell is empty, the function returns the row and column indices of that cell. If the end of the board is reached without finding any empty cells, the function returns (-1, -1) to indicate that the board is complete.

<br>

#### Board solver functions

8. `box_maker()`: This function creates 9 boxes that represent the 3x3 squares on a Sudoku board. It first creates a 3x3 grid of tiles, and then adds additional rows and columns to each box. It returns a list of lists, where each sublist contains the indices of the tiles in each box, sorted by their columns.

9. `check_board(board)`: This function takes a Sudoku board as input and checks if the board is a valid solution. It first creates a list of lists containing all the rows, columns, and 3x3 boxes on the board using box_maker(). Then, it checks if each row, column, and box contains all the numbers 1-9 without any duplicates. If any row, column, or box contains an asterisk ('*') or a duplicate number, the function returns False. Otherwise, it returns True.

10. `row_solve(board, change)`: This function takes a Sudoku board and a boolean change as inputs. It loops over each row of the board and checks if there is only one missing number in the row. If there is, it fills in the missing number with the only number that is not in the row, updates the board, and sets change to True. It then returns the updated board and the updated change value.

11. `column_solver(board, change)`: This function takes a Sudoku board and a boolean change as inputs. It loops over each column of the board and checks if there is only one missing number in the column. If there is, it fills in the missing number with the only number that is not in the column, updates the board, and sets change to True. It then returns the updated board and the updated change value.

12. `box_solver(boxes, board, change)`: This function takes the list of boxes created by box_maker(), a Sudoku board, and a boolean change as inputs. It loops over each box and checks if there is only one missing number in the box. If there is, it fills in the missing number with the only number that is not in the box, updates the board, and sets change to True. It then returns the updated board and the updated change value.

13. `hidden_single(boxes, board, change)`: This function takes the list of boxes created by box_maker(), a Sudoku board, and a boolean change as inputs. It loops over each box and each missing tile in each box. For each missing tile, it checks the row, column, and box that the tile is in to find the possible candidates for the missing number. If there is only one candidate, it fills in the missing number with the candidate, updates the board, and sets change to True. It then returns the updated board and the updated change value.

14. `soduko_solver(board)`: This function takes a Sudoku board as input and uses the above functions to solve the puzzle. It first creates the list of boxes using box_maker(). Then, it loops over the board and applies each solving strategy in turn until the board is solved. If the board is solved at any point, the function breaks out of the loop and returns the solved board.